# RegularExpressionGenerator

In [1]:
class NonDeterminsiticAutomaton:
    
    def __init__(self, q = None, sigma = None, delta = None, q0 = None, empty_symbol = "lambda",f = None):
        """
        Class initializar
        
        self.q receives all the states that conform the dfa
        
        self.sigma are all the variables defined in the dfa
        
        self.delta all are the operation defined, i.e- is a dictionary contating a tuple as key
        where the left values of the tuple is the current state and the right value the variable pass and the value obtain by
        accessing to with the key is to which state goes.
        
        self.q0 is the initial state
        
        self.empty_symbol is the symbol in which is represent the empty value, for instance, it is lambda bout could be epsilon
        or another symbol, but it has to be defined, it is not receive from txt.
        
        self.f are all the final states
        
        self.actual_state is the state which the dfa finish after proccessing a word, if no word proccess the actual state is q0
        and it is reset at start proccessing a new word
        """
        self.q = q
        self.sigma = sigma
        self.delta = delta
        self.q0 = q0
        self.empty_symbol = empty_symbol
        self.f = f
        self.actual_state = q0
        
        self.not_valid_actual_state = q0
        
        #private attributes
        self.set_empty_transictions = []
        
        self.all_reacheble_states = set()
        
        self.valid_paths = []
        
        self.not_valid_paths = []
        
        #boolean to know if a word is accepted or not
        self.trampa = False
        
        self.all_paths_for_a_symbol = []
        
        self.all_paths_empty_transitions = []
        
        
    def load_from_file(self, path, delta_separator = "-"):
        """
        Function to read a file that contains an automaton
        
        path: path file
        """
        
        #dictionary to save the parts of the automaton
        variables = {"q": None,
             "sigma": None,
             "delta": None,
             "q0": None,
             "f": None    
        }

        with open(path) as file:
            #parsing the document
            for line, variable in zip(file, variables):
                    line = line.replace("{", "")
                    line = line.replace("}", "")
                    line = line.replace("\n", "")
                    line = line.replace(" ", "")
                    value = list(line.split(","))
                    variables[variable] = value
                    
            #conditional to verify there is only one initial state        
            if len(variables["q0"]) > 1 or len(variables["q0"]) == 0:
                raise Exception('There are more than one initial state')
                
            variables["delta"] = [values.split(delta_separator) for values in variables["delta"]]
            
            #Memory to check that within the transictions there is no variable with more than one variable
            Memory = [True for value in variables["delta"] if value[1] not in variables["sigma"] and value[1] != self.empty_symbol]
            
            
            if sum(Memory) >= 1:
                # for example the language is {a, b} and the variable of that transiction is aa.
                raise TypeError("At least One variable has a transiction with a variable that does not belong to sigma")
                
            #creating all posible combination of each state with each symbol
            all_possible_transictions = {(state,symbol): ["z0_dead"] for state in variables["q"] for symbol in variables["sigma"] }
            
            #creating transitions from the state of dead
            for symbol in variables["sigma"]:
                 all_possible_transictions[("z0_dead",symbol)] = ["z0_dead"]
            
            

            #parsing
            #getting the values from delta
            for values in variables["delta"]:
                #creating the key
                key = (values[0],values[1])
                
                #checking whether the key is 
                if key in all_possible_transictions.keys():
                    #if the first variables is z0_dead, so that is the defaul creation and actually has transitions
                    if all_possible_transictions[key][0] == "z0_dead":
                        all_possible_transictions[key].pop()
                        all_possible_transictions[key].append(values[2])
                        
                    else:
                        all_possible_transictions[key].append(values[2])

                else:
                    all_possible_transictions[key] = [values[2]]
                        
             #getting q0
            variables["q0"] = variables["q0"][0]
            
            #assigning the attributos of the class to the values obtained from the file
            self.q = variables["q"]
            self.sigma = variables["sigma"]
            self.delta = all_possible_transictions
            self.q0 = variables["q0"] 
            self.f = variables["f"]
            self.actual_state = variables["q0"]
            
            #conditional to verify the dfa is valid
            if self.is_valid():
                pass
            else:
                raise Exception('Is not a valid automaton')
            
            
    
    def empty_transitions(self, state, showing_backtracking = True):
        """
        Function to find all reachable states from an initial state just passing empty symbols
        
        Input:
        Current State
        showing_backtracking: boolean to know whether to print the path or not
        
        Output:
        return a list of all reachable state
        """
        if set([state]).issubset(set(self.q)):
            pass
        else:
            raise Exception("The given state does not belong to this automaton")
        
        #reseting the set of empty transiction, this is a private attribute that is only useful in the methods of the classs
        self.set_empty_transictions = set()
        
        #creating a list to save the backtracking
        paths = [state]
        
        
        self.empty_transitions_recursive(state, [state], paths, showing_backtracking) 
        
        try:
            #if exists, then discard it
            self.set_empty_transictions.discard("z0_dead")
            return list(self.set_empty_transictions)
            
        except:                
            return list(self.set_empty_transictions)
        
        
        
    def empty_transitions_recursive(self, state, memory = [], paths = [], showing_backtracking = True):
        """
        Private Function to find all reachable states from an initial state just passing empty symbols, recursively
        we did it in this way so we can get print the path, I mean for empty transitions, getting the paths may be
        useless because it works for nothing but it seems neat xD.
        
        Input:
        state: Current State
        memory: is a list of states already visited by that path
        showing_backtracking: boolean to know whether to print the path or not
        
        Output:
        return a list of all reachable state
        """
    
        #memory to avoid infinite loops
        memory_inside = memory[:]
        
        memory_inside.append(state)
        
        if showing_backtracking:
            print(paths)
        
        #check if that transiction exists
        try:
            for new_state in self.delta[(state, self.empty_symbol)]:
                #to avoid infinite loops
                if new_state not in memory:
                    #backtracking
                    paths.append(new_state)
                    
                    #adding the new_state into that set
                    self.set_empty_transictions.add(new_state)
                    #recursion 
                    self.empty_transitions_recursive(new_state, memory_inside, paths, showing_backtracking)
                    
                    #eliminating the element, for backtracking
                    paths.pop()
                    
                else:
                    #when the infinite loop exist
                    paths.append(new_state)
                    
                    if showing_backtracking:
                        print(paths)
                        
                    #eliminating the element, for backtracking
                    paths.pop()
                    
        except:
            pass
            
        
        
        
    def process_symbol(self, state, symbol, showing_backtracking = True):
        """
        Function to find all the states you can reach from one state processing a symbol
        
        Input:
        state: current state
        
        symbol: variable belonging to sigma
        
        showing_backtracking: boolean to know whether to print the path or not
        
        Output:
        return a list of the reacheble states
        """
        self.valid_string(symbol)
        
        if set([state]).issubset(set(self.q)):
            pass
        else:
            raise Exception("The given state does not belong to this automaton")
        
        #reseting attribute
        self.all_paths_for_a_symbol = []
        
        #setting that set into the first reachable states
        #First case where only consumes the symbol
        self.all_reacheble_states  = set(self.delta[(state, symbol)])
        
        paths = [(state, symbol)]
        
        paths.append(self.delta[(state, symbol)])
        if showing_backtracking:
            print("-------------------------Case where reach a state only consuming the symbol-------------------------")
            print(paths)
            
        if self.delta[(state, symbol)] != []:
            self.all_paths_for_a_symbol.append(paths[:])
            
        paths.pop()
        
        
        #Second case where the symbol is consumed and then applyes lambda
        
        if showing_backtracking:
            print("-------------------------Case where reach a state consuming the symbol and then lambda if possible-------------------------")
        
        for new_state in self.delta[(state, symbol)]:
            
            if new_state != "z0_dead":
                #adding the state and the symbol for transiction
                paths.append((new_state, self.empty_symbol))
                

                #adding where it can reach after making that transiction
                empty_transitions_list = self.empty_transitions(new_state, False)
                
                paths.append(empty_transitions_list)
                
                if empty_transitions_list != []:
                    self.all_paths_for_a_symbol.append(paths[:])
                if showing_backtracking:
                    print(paths)

                #uptading the set
                self.all_reacheble_states.update(empty_transitions_list)
                paths.pop()
                paths.pop()
        
        
        #third case first consumbes the empty symbol and then the synbol 
        #and also fourth case where consumes the empty symbol, then the symbol and again the empty symbol
        #getting the transitions using lambda
        new_reacheable_states = self.empty_transitions(state, False)
        
        
        if showing_backtracking:
            print("-------------------------Third and Fourth Case-------------------------")
        
        paths = [(state, self.empty_symbol)]
        
        #going through the reacheable states using lambda and then the symbol
        for new_state in new_reacheable_states:
            
            #saving the symbol and the symbol consume
            paths.append((new_state, symbol))
            #adding the reacheable states
            paths.append(self.delta[(new_state, symbol)])
            if showing_backtracking:
                print(paths)
    
            self.all_reacheble_states.update(self.delta[(new_state, symbol)])
            
            self.all_paths_for_a_symbol.append(paths[:])
            #eliminating those reachebale states because where are going to iterate, one by one
            paths.pop()
            
            #going through the reacheable states using lambda, then the symbol and finally the empty transition
            for second_next_state in self.delta[(new_state, symbol)]:
                if second_next_state != "z0_dead":
                    #saving the symbol and the symbol consume
                    paths.append((second_next_state, self.empty_symbol))
                    #adding the reacheable states
                    
                    empty_transitions_list = self.empty_transitions(second_next_state, False)
                    
                        
                    paths.append(empty_transitions_list)
                    if showing_backtracking:
                        print(paths)
                    
                    if empty_transitions_list != []:
                        self.all_paths_for_a_symbol.append(paths[:])
    
                    self.all_reacheble_states.update(empty_transitions_list)

                    #eliminating for backtracking
                    paths.pop()
                    paths.pop()

            #eliminating for backtracking    
            paths.pop()
            
        lista_reacheable_state = list(self.all_reacheble_states)    
            
        if "z0_dead" in lista_reacheable_state:
            lista_reacheable_state.append(lista_reacheable_state.pop(lista_reacheable_state.index("z0_dead")))
            
        return lista_reacheable_state
    
    def process_word(self, word, showing_backtrack = True):
        """
        Function to determine if a word belong to the language of the ndfa
        
        input:
        Word: word to be proccessed
        
        Output:
        return a bolean whether if the word is proccessed by the automaton or not
        """ 
        self.valid_string(word)
        
        word = str(word)
        
        #reseting attributes
        self.trampa = False

        self.valid_paths = []

        self.not_valid_paths = []

        #memory
        paths = []
        
        if word == "":
            for state in self.empty_transitions(self.q0,False):
                if state in self.f:
                    self.trampa = True
                    self.valid_paths.extend([(self.q0, self.empty_symbol), state])
                    return self.trampa
         
        else:
            
            #going into recursive
            self.process_word_recursive(word, self.q0 ,paths, showing_backtrack)

            return self.trampa
        
    
    def process_word_recursive(self, word, state,paths = [], showing_backtrack = True):
        """
        Private method for the method process_word, in which it recursively goes to each possible paths to try to find a possible path 
        where the word is accepted
        input:
        Word: word to be proccessed
        paths: list which is the memory for the backtracking
        showing_backtrack: shows the backtracking or not
        
        Output:
        return a bolean whether if the word is proccessed by the automaton or not
        """
        #check if the word is not empty
        try:
            letter = word[0]
        
        #case where is no more variables to consume
        except:
            #checking if the last state is in final states
            if state in list(self.f):
                if showing_backtrack:
                    print("---------------------------valid-----------------------\n\n")
                #saving that the word is valid   
                self.trampa = True
                #actul state to use in function str
                self.actual_state = state
                #saving the valid path
                self.valid_paths.append(paths[:])
                return 
            
            #unvalid word
            else:
                if showing_backtrack:
                    print("---------------------------Unvalid---------------------------\n\n")
                    #the same as the part latter
                self.not_valid_actual_state = state
                self.not_valid_paths.append(paths[:])
                return
        
        #adding to the backtracking
        paths.append((state,letter))
        if showing_backtrack:
            print(paths)
        #going through each possible state with that letter
        for possible_path in self.process_symbol(state, letter, False):
            #saving for backtracking
            paths.append(possible_path)
            if showing_backtrack:
                print(paths)
                
            #if it reaches the state of dead
            if possible_path == "z0_dead":
                if showing_backtrack:
                    print("---------------------------Unvalid---------------------------\n\n")
                paths.pop()
                paths.pop()
                #you may think this should be a pass, but no because our  state of death, thank to the function process_symbol
                # is always and the end of the list
                return
            
            self.process_word_recursive(word[1:], possible_path, paths, showing_backtrack)
            paths.pop()
        paths.pop()
        
    def valid_string(self, word):
        
        if set(word).issubset(set(self.sigma)):
            return
        else:
            raise Exception("Not valid word, contains variables that are not part of the language")
    
        
        
    def __str__(self):
        
        if len(self.valid_paths) == 0:
            self.actual_state = self.not_valid_actual_state
        
        cadena = "Grammar: \n q: {} \n sigma: {}, delta: {}, q0: {}, f: {}, current_sate: {}, empty_symbol: {}".format(
                                                                                    self.q , 
                                                                                    self.sigma, 
                                                                                    self.delta, 
                                                                                    self.q0, 
                                                                                    self.f, 
                                                                                    self.actual_state,
                                                                                    self.empty_symbol) 
        return cadena
    
    def is_valid(self):
        
        """
        Funtion to validate whether the dfa is valid or not
        
        Uses the attributes of the class
        """
        
        #gets the tuples which are the deltas defined
        all_defined_oper = [(key1,key2)for key1, key2 in self.delta.keys() if key1 != "z0_dead"]
        
        #get the values of the deltas
        all_to_go_states = [value for value in self.delta.values()]
        
        all_to_go_states = [item for sublist in all_to_go_states for item in sublist if item != "z0_dead"]
        
        #gets the left value of the tuple
        all_states_to_receive = set([state[0] for state in list(self.delta.keys()) if state[0] != "z0_dead"])
        
        #gets the right value of the tuple
        all_variables_to_receive = set([state[1] for state in list(self.delta.keys()) if state[0] != "z0_dead"])
        
        
        #conditional to check if the final states are subsets of q, which is the variable containing all the states
        if set(self.f).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("States of F are not a subset of states q")
            
            
        #conditional to check if the inital state is a subsets of q, which is the variable containing all the states    
        if set([self.q0]).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("q0 is not a subset of states q")
            
        
        #checking if the left value of the tuple is subset of q and the right value is subset of sigma
        if set(all_states_to_receive).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("The state before a applying the transiction is not a subset of q")
            
        sigmas = set(self.sigma)
        sigmas.add(self.empty_symbol)
        if set(all_variables_to_receive).issubset(sigmas):
            pass
        
        else:
            print(all_variables_to_receive)
            print(self.sigma)
            raise TypeError("The variable which is apply to the transiction is not a subset of sigma")
        
        #checking if all the transictions of delta are subsets of q
        if set(all_to_go_states).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("The state resultant of a transiction is not a subset of states q")
            
        return True
    
    def print_valid_path(self, path = None):
        """
        Function to print one path of an accepted word, only works with valid paths, can be obtained using self.valid_paths
        
        input:
        path: list that contains the path travelled
        
        output:
        None
        only prints the path
        """
        # if the path is not defined, it gets the first path of valid_paths
        if path == None:
            path = self.valid_paths[0]
            
        #saving the paths
        llaves = []
        caminos = []
        
        #this division is used to avoid problems and make it easier to use because a valid path always has a pair number of elements+
        #where the pair are a tuple of the transiction and the impairs is the resulf ot the tuple before
        n = int(len(path)/2)
        
        #going through this list
        for i in range(n):
            #process the symbol of the right side of the tuple in the state that is in the left side of the same tuple
            self.process_symbol(path[i*2][0],path[i*2][1], False)
            
            #going through each set of paths
            for j in range(len(self.all_paths_for_a_symbol)):
                #the list obtained in process symbol and saved in self.all_paths_for_a_symbol is made to each last element of the list within 
                #list are the states that reaches, so we get a path to reach that state consuming that symbol
                if path[i*2 + 1] in self.all_paths_for_a_symbol[j][-1]:
                    aux = self.all_paths_for_a_symbol[j]
                    #saving the values
                    llaves.append((path[i*2][0],path[i*2][1], path[i*2 + 1])) 
                    caminos.append(aux)
                    break 
        
        #we eliminate if there is more than one final state in the last element of the last list
        for i in caminos[-1][-1]:
            if i in self.f:
                caminos[-1][-1] = [i]
                break
        
        try:
            #This code is to clean the paths adding the exact lambda transiction
            n = len(caminos)
            for path in range(n):
                for trans in range(len(caminos[path])):
                    #all the case before getting to the last element that always is an array
                    if trans != len(caminos[path])-1:
                        #case where the transiction is lambda
                        if caminos[path][trans][1] == self.empty_symbol:
                            #finding the path
                            self.find_path(caminos[path][trans][0], caminos[path][trans+1][0])
                            #case where the transiction is before the last element
                            if trans == len(caminos[path])-2:
                                caminos[path].pop(trans)
                                caminos[path].pop(trans)
                                #adding the transictions using lambda to get to the last element of the list
                                for val in self.all_paths_empty_transitions[::-1]:
                                        caminos[path].insert(trans, val)  
                            #case where the element after the transictions is another tuple
                            else:

                                caminos[path].pop(trans)
                                #adding the elements except the last one because is just a list, and is worthless
                                for val in self.all_paths_empty_transitions[1::-1][1:]:
                                        caminos[path].insert(trans, val) 
                                        
            i = 1
            #loops where the printins is made
            j = 0
            for values in list(caminos):
                print("{}.- ".format(i), end = "")
                for transition in range(len(values)):
                    if transition < (len(values)-1):
                        if len(values[transition+1]) > 1 and type(values[transition+1]) == list:
                            for val in values[transition+1]:
                                if j < len(caminos) - 1:
                                    if val in caminos[j+1][0]:
                                        print("{}  ======> {}---- ".format(values[transition],val), end = "")  
                                else:
                                    if val in self.f:
                                        print("{}  ======> {}---- ".format(values[transition],val), end = "")
                                        break
                        else:
                            print("{}  ======> {}---- ".format(values[transition],values[transition+1][0]), end = "")

                    else:
                        print()
                j += 1
                i += 1
                            
        except:
            self.valid_paths = self.valid_paths[1:]
            self.print_valid_path()
            
        
    
    def find_path(self, state, reach_state):
        """
        Function to find a path from a state to other state using only lambda
        
        Input:
        state: initial state
        
        reach_state: state where we want to reach
        
        Output: One possible path from the state to the reach_state using empty_symbols
        
        """
        self.all_paths_empty_transitions = []

        self.__find_path(state, reach_state, memory = [], anti_loops = [])
        
        self.all_paths_empty_transitions = self.all_paths_empty_transitions[0]

    def __find_path(self, state, reach_state, memory = [], anti_loops = []):
        """
        Private function for find_path
        Recursively gets all the paths from state to reach_state
        
        """
        anti_loops.append(state)
        #case to finish recursivity
        if state == reach_state:
            memory.append([state])
            self.all_paths_empty_transitions.append(memory[:])
            return
        else:
            memory.append((state, self.empty_symbol))
            try:
                #checking if there is transiction with the empty_symbol
                valores = self.delta[state,self.empty_symbol]
                for estado in valores:
                    if estado not in anti_loops:
                        self.__find_path(estado, reach_state, memory, anti_loops)
                    memory.pop()
                    
            except:
                pass

In [2]:
import re, copy

class RegularExpressionGenerator(NonDeterminsiticAutomaton):
    
    def __init__(self, empty_symbol = "lambda"):
        super().__init__(empty_symbol = empty_symbol)
        
        self.re_expressions = dict()
        
        self.re_compiler = dict()
        
        """
        Note the following attributes are not necessary because for educational porpouse allow us to see the initial of the automaton
        and step by step how are pull together, however, they can be eliminated and the code does not seems affect, but the lines of code,
        where those attributes appear have to be eliminated too. Since this is a educational porpouse we decided to not eliminate them but if 
        for the grade is necessary memory efficency, we can change it.
        
        """
        
        self.re_expressions_copy = dict()
        
        self.re_compiler_copy = dict()
        
        self.re_expressions_aux = dict()
        
        self.re_compiler_aux = dict()
        
        
        #these attributes is necessary
        self.remaining_states = []
        
        self.delta2 = dict()
        
    
    def __create_new_start_final_state(self):
        """
        Create the new initial state, connecting this one to the before initial state using the empty transition, and also creates a new final
        state, and the previous final states are connected to this one using the empty transition
        """        
        
        self.delta2[("new_q0",self.empty_symbol)] = [self.q0[:]]
        
        for state in self.f:
            try:
                self.delta2[(state,self.empty_symbol)].append("new_final")
                
            except:
                self.delta2[(state,self.empty_symbol)] = ["new_final"]
                
        
    
    def __initializing_re(self):
        """
        Private method to initialie both re expression and re compile
        
        return or rather assigns the initial values for each re 
        """
        
        for transiction in self.delta2:
            for state in self.delta2[transiction]:
                
                #remember z0_dead is a state created by us, so in the reality is not part of the automaton
                if state != "z0_dead" and transiction[0] != "z0_dead":
                    if transiction[1] == self.empty_symbol:
                        aux = "λ"

                    else:
                        aux = transiction[1]
                        
                    #parsing, if want to see what it does, check the attribute re_expressions_aux and re_compiler_aux
                    try:
                        tempor = self.re_compiler[(transiction[0], state)]
                        if "λ" in tempor:
                            self.re_compiler[(transiction[0], state)] = "(" + str(aux) + ")?"
                        
                        elif "λ" in str(aux):
                            if "?" in tempor:
                                pass
                            else:
                                self.re_compiler[(transiction[0], state)] = "(" + tempor + ")?"
                        
                        else:
                            if "?" in self.re_compiler[(transiction[0], state)]:
                                self.re_compiler[(transiction[0], state)] = self.re_compiler[(transiction[0], state)][1:-2] + str(aux) + ")?"
                            else:
                                self.re_compiler[(transiction[0], state)] += "|"  + str(aux) 

                        self.re_expressions[(transiction[0], state)] += " U " + str(aux)

                    except:
                        self.re_compiler[(transiction[0], state)] =  str(aux) 

                        self.re_expressions[(transiction[0], state)] = str(aux)
                        

                
    
    def __in_out_lists(self, state, in_list = [], out_list = []):
        """
        Function to create the list in and out, in are the states that can get to the given state and out is the states reacheables by that
        state
        
        Input:
        state: state is needed the in and out lists
        in_list: list where to save the values for the in states
        out_list: list where to sabe the values for the out states
        
        Output:
        return in_list and out_list
        """
        for transiction in self.re_expressions:
            if transiction[0] == state and transiction[1] != state:
                out_list.append(transiction[1])
                
            elif transiction[1] == state and transiction[0] !=state:
                in_list.append(transiction[0])
    
    def __permutation(self, list1 = [], list2 = []):
        """
        Function to create all the combinations between all the values of two given lists
        
        input:
        list1: list 
        list2: list
        
        output:
        return a list containing the combinations of both lists
        """
        combination = []
        
        for i in list1:
            for j in list2:
                combination.append((i,j))
        
        return combination
    
    
    def __eliminate_key(self, state):
        """
        Function to eliminate garbage values given a state
        
        Input:
        state to eliminate the key and the values in the dictionary
        """
        aux = self.re_expressions.copy()
        for states in aux:
            if state == states[0] or state == states[1]:
                self.re_expressions.pop(states, None)
                
        aux = self.re_compiler.copy()
        
        for states in aux:
            if state == states[0] or state == states[1]:
                self.re_compiler.pop(states, None)
    
    def convert_automaton(self):
        """
        Function to get the regular expression seen in class, and the regular expression for the re python library
        
        return those re expression
        
        """
        #copy of delta
        self.delta2 = copy.deepcopy(self.delta)
        
        #creating the new and final states
        self.__create_new_start_final_state()
        
        #initilazing the re dictionaries
        self.__initializing_re()
        
        self.re_expressions_copy = copy.deepcopy(self.re_expressions)
        
        self.re_expressions_aux = copy.deepcopy(self.re_expressions)
        
        self.re_compiler_copy = copy.deepcopy(self.re_compiler)
        
        self.re_compiler_aux = copy.deepcopy(self.re_compiler)
        
        #loop to go through all the states of the automata
        for state in self.q:
            #initializing and reseting the in and out list
            in_list = []
            out_list = []
            
                
            #creating the in and out lists
            self.__in_out_lists(state, in_list, out_list)

            #creating the combinations of those states
            combinations = self.__permutation(in_list, out_list)

            #going through each comb
            for comb in combinations:

                """
                Part for regular expression
                """

                aux1 = self.re_expressions[(comb[0],state)]
                aux2 = self.re_expressions[(state, comb[1])]

                """
                All the following code all the if, elif, try and excepts is just to give that neat format
                """
                if len(aux1) > 1 and "(" not in aux1 and "U" in aux1:
                    aux1 = "(" + aux1 + ")"

                if len(aux2) > 1 and "(" not in aux2 and "U" in aux2:
                    aux2 = "(" + aux2 + ")"


                #checking if the current state has a transiction to itself
                try:
                    temp = self.re_expressions[(state,state)]

                    #case where it has already the parenthesis
                    if temp[0] == "(" and temp[-1] == ")":
                        aux3 = temp + "*"

                    else:
                        aux3 = "(" + temp + ")*"

                #if not is empty
                except:
                    aux3 = ""

                #the try is to check if the key already exists
                if comb in self.re_expressions:
                    #case where the state has a transiction to itself
                    if aux3 != "":
                        self.re_expressions[comb] = "("+self.re_expressions[comb]+" U "+"("+aux1 + "." + aux3 + "." + aux2 + ")" +")"
                        self.re_expressions_copy[comb] = self.re_expressions[comb]

                    else:
                        self.re_expressions[comb] = "(" + self.re_expressions[comb] + " U " + aux1 + "." + aux2 + ")"
                        self.re_expressions_copy[comb] = self.re_expressions[comb]

                else:

                    #case where the state has a transiction to itself
                    if aux3 != "": 
                        if aux1 == "λ" and aux2 == "λ" and aux3 != "(λ)*":
                            self.re_expressions[comb] = aux3
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        elif aux1 == "λ" and aux2 != "λ" and  aux3 != "(λ)*":
                            self.re_expressions[comb] = aux3 +  "." + aux2 
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        elif aux2 == "λ" and aux1 != "λ" and  aux3 != "(λ)*":
                            self.re_expressions[comb] = aux1 + "." + aux3 
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        elif aux3 == "(λ)*" and aux1 == "λ" and aux2 == "λ":
                            self.re_expressions[comb] = "λ"
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        elif aux3 == "(λ)*" and aux1 == "λ" and aux2 != "λ":
                            self.re_expressions[comb] = aux2
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        elif aux3 == "(λ)*" and aux1 != "λ" and aux2 == "λ":
                            self.re_expressions[comb] = aux1 
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        elif aux3 == "(λ)*" and aux1 != "λ" and aux2 != "λ":
                            self.re_expressions[comb] = aux1 + "." + aux2
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        else:
                            self.re_expressions[comb] = aux1 + "." + aux3 +  "." + aux2
                            self.re_expressions_copy[comb] = self.re_expressions[comb]
                    else:

                        if aux1 == "λ" and aux2 == "λ":
                            self.re_expressions[comb] = "λ" 
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        elif aux1 == "λ" and aux2 != "λ":
                            self.re_expressions[comb] = aux2 
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        elif aux2 == "λ" and aux1 != "λ":
                            self.re_expressions[comb] = aux1
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                        else:
                            self.re_expressions[comb] = aux1 + "." + aux2
                            self.re_expressions_copy[comb] = self.re_expressions[comb]

                """
                Part for the expression for the re library
                """
                aux1 = self.re_compiler[(comb[0],state)]
                aux2 = self.re_compiler[(state, comb[1])]

                if len(aux1) > 1 and "(" not in aux1 and "|" in aux1:
                    aux1 = "(" + aux1 + ")"

                if len(aux2) > 1 and "(" not in aux2 and "|" in aux2:
                    aux2 = "(" + aux2 + ")"
                    
                if aux1 == "λ":
                    aux1 = ""

                if aux2 == "λ":
                    aux2 = ""

                #try to check if the state has a transiction to itself
                try:
                    temp = self.re_compiler[(state,state)]
                    
                    aux3 = temp + "*"

                    #if temp[0] == "(" and temp[-1] == ")":
                     #   aux3 = temp + "*"
                    #else:
                     #   aux3 = "(" + temp + ")*"

                except:
                    aux3 = ""

                #try to check if exist the key comb
                if comb in self.re_compiler:
                    #exist a transiction to itself
                    if aux3 != "":
                        tempor = self.re_compiler[comb]

                        if tempor == "":
                            if aux1 == "" and aux3 == "" and aux2 == "":
                                self.re_compiler[comb] =  ""
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            else:
                                self.re_compiler[comb] =  "(" + aux1 + aux3 + aux2 + ")?"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                        elif aux1 == "":
                            if aux3 == "" and aux2 == "":
                                self.re_compiler[comb] = self.re_compiler[comb]
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            else:
                                self.re_compiler[comb] = "(" + self.re_compiler[comb] +"|" + "(" + aux3 + aux2 + ")" + ")"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                        elif aux2 == "":
                            if aux1 == "" and aux3 == "":
                                self.re_compiler[comb] = self.re_compiler[comb]
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            else:
                                self.re_compiler[comb] = "(" + self.re_compiler[comb] +"|" + "(" + aux1 + aux3 + ")" + ")"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                        elif aux3 == "":
                            if aux1 == "" and aux2 == "":
                                self.re_compiler[comb] = self.re_compiler[comb]
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            else:
                                self.re_compiler[comb] = "(" + self.re_compiler[comb] +"|" + "(" + aux1 + aux2 + ")" + ")"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                        else:
                            if aux1 == "" and aux2 == "" and aux3 == "":
                                self.re_compiler[comb] = "(" + self.re_compiler[comb] +"|" + "(" + aux1 + aux3 + aux2 + ")" + ")"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            else:
                                self.re_compiler[comb] = "(" + self.re_compiler[comb] +"|" + "(" + aux1 + aux3 + aux2 + ")" + ")"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                    #there no exists transiction to itself
                    else:
                        tempor = self.re_compiler[comb]
                        if tempor == "":
                            if aux1 == "" and aux2 == "":
                                self.re_compiler[comb] = ""
                                self.re_compiler_copy[comb] = self.re_compiler[comb]


                            else:
                                self.re_compiler[comb] = "(" + aux1 + aux2 + ")?"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                        elif aux1 == "":
                            if aux2 == "":
                                self.re_compiler[comb] = self.re_compiler[comb]
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            else:
                                self.re_compiler[comb] = "(" + self.re_compiler[comb] + "|" + aux2 + ")"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                        elif aux2 == "":
                            if aux1 == "":
                                self.re_compiler[comb] = self.re_compiler[comb]
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            else:
                                self.re_compiler[comb] = "(" + self.re_compiler[comb] + "|" + aux1 + ")" 
                                self.re_compiler_copy[comb] = self.re_compiler[comb]
                                    
                        else:    
                            if aux1 == "" and aux2 == "":
                                self.re_compiler[comb] = self.re_compiler[comb]
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            else:
                                self.re_compiler[comb] = "(" + self.re_compiler[comb] + "|" + "(" +aux1 + aux2 + ")" + ")"
                                self.re_compiler_copy[comb] = self.recompiler[comb]

                else:
                    #exist a transiction to itself
                    if aux3 != "": 
                        if aux1 == "λ":
                            aux1 = ""

                        if aux2 == "λ":
                            aux2 = ""

                        if aux3 == "(λ)*":
                            aux3 = ""


                        if aux1 == "" and aux2 == "" and aux3 == "":
                            self.re_compiler[comb] = ""
                            self.re_compiler_copy[comb] = self.re_compiler[comb]

                        else:
                            if aux1 != "" and aux2 == "" and aux3 == "":
                                self.re_compiler[comb] = aux1
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            elif aux1 == "" and aux2 != "" and aux3 == "":
                                self.re_compiler[comb] = aux2
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                            elif aux1 == "" and aux2 == "" and aux3 != "":
                                self.re_compiler[comb] = aux3
                                self.re_compiler_copy[comb] = self.re_compiler[comb]
                            else:
                                self.re_compiler[comb] = "(" + aux1 + aux3 + aux2 + ")"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]

                    #there no exists transiction to itself
                    else:
                        if aux1 == "λ":
                            aux1 = ""

                        if aux2 == "λ":
                            aux2 = ""

                        if aux1 == "" and aux2 == "":
                            self.re_compiler[comb] = ""
                            self.re_compiler_copy[comb] = self.re_compiler[comb]

                        else:
                            if aux1 == "":
                                self.re_compiler[comb] = aux2
                                self.re_compiler_copy[comb] = self.re_compiler[comb]  

                            elif aux2 == "":
                                self.re_compiler[comb] = aux1
                                self.re_compiler_copy[comb] = self.re_compiler[comb]  

                            else:
                                self.re_compiler[comb] = "(" + aux1 + aux2 + ")"
                                self.re_compiler_copy[comb] = self.re_compiler[comb]  



            self.__eliminate_key(state)
                
                
    def check_word(self, string, printing = True):
        """
        Function to find all matches of the expression in the string.
        
        Input:
        given string to find the matches
        
        Output:
        return all matches of the expression in the string
        """
        
        ans = []
        exist_lambda = False
        
        #checking if the state can reach the final state being an empty string or ""
        for state in self.empty_transitions(self.q0,False):
            if state in self.f:
                exist_lambda = True
                break
        
        #checking if the convert automaton is already run
        if ('new_q0', 'new_final') in self.re_compiler:
            
            #getting the pattern
            pattern = self.re_compiler[('new_q0', 'new_final')]

            match = re.findall(pattern, string)
            for match in re.finditer(pattern, string):
                if match.group() == "":
                    #case where the "" is accepted in the automaton
                    if exist_lambda:
                        
                        if printing: print(match.group())
                        ans.append(match.group())
                        
                    else:
                        pass
                        
                else:
                    if printing: print(match.group())
                    ans.append(match.group())
        
        #case where convert automaton have not been run
        else:
            
            self.convert_automaton()
            
            try:
                pattern = self.re_compiler[('new_q0', 'new_final')]
                
            except:
                raise Exception("This automata its final states are not reacheable, so there is not a regular expression")

            match = re.findall(pattern, string)
            for match in re.finditer(pattern, string):
                if match.group() == "":
                    #case where the "" is accepted in the automaton
                    if exist_lambda:
                        print(match.group())
                        ans.append(match.group())
                        
                    else:
                        pass
                        
                else:
                    print(match.group())
                    ans.append(match.group())
                
        return ans

# Code explanation

As we have gone through the explanation of NonDeterminsiticAutomaton, we are not going to go into its methods, we are only going to focus on the methods of the new class class RegularExpressionGenerator

## Private Method: __create_new_start_final_state

```
def __create_new_start_final_state(self):

        self.delta2[("new_q0",self.empty_symbol)] = [self.q0[:]]
        
        for state in self.f:
            try:
                self.delta2[(state,self.empty_symbol)].append("new_final")
                
            except:
                self.delta2[(state,self.empty_symbol)] = ["new_final"]

```

As its name says, this function create the new initial state which is connected to the prior initial state using an empty transition, and creates the final states which is reached by all the previous final states using an empty transition. And the function also add the respectives transition for those new states to reach the void state 

## Private Method: __initializing_re

```
def __initializing_re(self):
        
        for transiction in self.delta2:
            for state in self.delta2[transiction]:
                
                if state != "z0_dead" and transiction[0] != "z0_dead":
                    if transiction[1] == self.empty_symbol:
                        aux = "λ"

                    else:
                        aux = transiction[1]
                        
                    try:
                        tempor = self.re_compiler[(transiction[0], state)]
                        if "λ" in tempor:
                            self.re_compiler[(transiction[0], state)] = "(" + str(aux) + ")?"
                        
                        elif "λ" in str(aux):
                            self.re_compiler[(transiction[0], state)] = "(" + tempor + ")?"
                        
                        else:
                            self.re_compiler[(transiction[0], state)] += "|"  + str(aux) 

                        self.re_expressions[(transiction[0], state)] += " U " + str(aux)

                    except:
                        self.re_compiler[(transiction[0], state)] =  str(aux) 

                        self.re_expressions[(transiction[0], state)] = str(aux)
                        
```

__initialiing_re what it does is to create the first instances for both the re (python library) expression and the regular expression (terms seen in class), it makes a litle change the keys are the two connected states, the left state is the input state and the right the output state, so the values of the key are the actual expressions for that transaction.

For example, we have the transicionts ("q0","1") = q1 and ("q0","0") = q1, this becomes

("q0","q1") = 0 U 1, for regular expression and ("q0","q1") = 0 | 1, for re compiler



## Private method in_out_lists

```
def __in_out_lists(self, state, in_list = [], out_list = []):

        for transiction in self.re_expressions:
            if transiction[0] == state and transiction[1] != state:
                out_list.append(transiction[1])
                
            elif transiction[1] == state and transiction[0] !=state:
                in_list.append(transiction[0])

```

This function gets a list of all the states that can reach a given state , in_list, and gets a list of all the reacheable states for a given state, out_list.

## Private method permutation

```
def __permutation(self, list1 = [], list2 = []):
        combination = []
        
        for i in list1:
            for j in list2:
                combination.append((i,j))
        
        return combination
```

This function gets all the possible pair combinations between each elements of two given lists:

Example given the lists [1,2,3] and [4,5].

It will return [(1,4),(1,5), (2,4), (2,5), (3,4) , (4,5)]

## Private method eliminate_key

```
def __eliminate_key(self, state):
        aux = self.re_expressions.copy()
        for states in aux:
            if state == states[0] or state == states[1]:
                self.re_expressions.pop(states, None)
                
        aux = self.re_compiler.copy()
        
        for states in aux:
            if state == states[0] or state == states[1]:
                self.re_compiler.pop(states, None)
                
```

This function eliminates all the transiction in both re_expressions and re_compiler of a given states, this state is already used, therefore, having it still in the transiction makes an overlap

## Private method check_word

```
def check_word(self, string):        
        if ('new_q0', 'new_final') in self.re_compiler:
            
        
            pattern = self.re_compiler[('new_q0', 'new_final')]

            match = re.findall(pattern, string)
            for match in re.finditer(pattern, string):
                print(match.group())
        else:
            
            self.convert_automaton()
            
            pattern = self.re_compiler[('new_q0', 'new_final')]

            match = re.findall(pattern, string)
            for match in re.finditer(pattern, string):
                print(match.group())
```

The function gets all the matchs on a string of the re expression of the automaton

### Note: The function convert_automaton we consider is well explained in the code comments and even though contains a lot of try - except and if - else, is not that complicated because that is to only give it format. And the method is understandable by its own.

# Implementation 1

In [3]:
ndfa = RegularExpressionGenerator(empty_symbol = "lambda")
ndfa.load_from_file("automaton_examples/automata.txt")

In [4]:
ndfa.convert_automaton()

In [5]:
ndfa.re_compiler_aux

{('q0', 'q3'): '0',
 ('q0', 'q1'): '0',
 ('q1', 'q2'): '0',
 ('q1', 'q1'): '1',
 ('q2', 'q2'): '0',
 ('q2', 'q1'): '(1)?',
 ('q3', 'q3'): '0',
 ('q0', 'q2'): 'λ',
 ('q2', 'new_final'): 'λ',
 ('new_q0', 'q0'): 'λ'}

In [6]:
ndfa.re_compiler_copy

{('q0', 'q3'): '0',
 ('q0', 'q1'): '0',
 ('q1', 'q2'): '0',
 ('q1', 'q1'): '1',
 ('q2', 'q2'): '(0|((1)?1*0))',
 ('q2', 'q1'): '(1)?',
 ('q3', 'q3'): '0',
 ('q0', 'q2'): 'λ',
 ('q2', 'new_final'): 'λ',
 ('new_q0', 'q0'): 'λ',
 ('new_q0', 'q3'): '0',
 ('new_q0', 'q1'): '0',
 ('new_q0', 'q2'): '(01*0)?',
 ('new_q0', 'new_final'): '((01*0)?(0|((1)?1*0))*)'}

In [7]:
ndfa.re_compiler

{('new_q0', 'new_final'): '((01*0)?(0|((1)?1*0))*)'}

In [8]:
ndfa.re_expressions_aux

{('q0', 'q3'): '0',
 ('q0', 'q1'): '0',
 ('q1', 'q2'): '0',
 ('q1', 'q1'): '1',
 ('q2', 'q2'): '0',
 ('q2', 'q1'): '1 U λ',
 ('q3', 'q3'): '0',
 ('q0', 'q2'): 'λ',
 ('q2', 'new_final'): 'λ',
 ('new_q0', 'q0'): 'λ'}

In [9]:
ndfa.re_expressions_copy

{('q0', 'q3'): '0',
 ('q0', 'q1'): '0',
 ('q1', 'q2'): '0',
 ('q1', 'q1'): '1',
 ('q2', 'q2'): '(0 U ((1 U λ).(1)*.0))',
 ('q2', 'q1'): '1 U λ',
 ('q3', 'q3'): '0',
 ('q0', 'q2'): 'λ',
 ('q2', 'new_final'): 'λ',
 ('new_q0', 'q0'): 'λ',
 ('new_q0', 'q3'): '0',
 ('new_q0', 'q1'): '0',
 ('new_q0', 'q2'): '(λ U (0.(1)*.0))',
 ('new_q0', 'new_final'): '(λ U (0.(1)*.0)).(0 U ((1 U λ).(1)*.0))*'}

In [10]:
ndfa.re_expressions

{('new_q0', 'new_final'): '(λ U (0.(1)*.0)).(0 U ((1 U λ).(1)*.0))*'}

## Inherited methods

In [11]:
ndfa.process_word("010101000", False)

True

In [12]:
ndfa.print_valid_path()

1.- ('q0', 'lambda')  ======> q2---- ('q2', '0')  ======> q2---- 
2.- ('q2', '1')  ======> q1---- 
3.- ('q1', '0')  ======> q2---- 
4.- ('q2', '1')  ======> q1---- 
5.- ('q1', '0')  ======> q2---- 
6.- ('q2', '1')  ======> q1---- 
7.- ('q1', '0')  ======> q2---- 
8.- ('q2', '0')  ======> q2---- 
9.- ('q2', '0')  ======> q2---- 


In [13]:
ndfa.process_word("", False)

True

In [14]:
ndfa.valid_paths

[('q0', 'lambda'), 'q2']

### re_compiler

In [15]:
string = "01010100021000" 
ans = ndfa.check_word(string, False)

In [16]:
for i in ans:
    print(ndfa.process_word(i, False))

True
True
True
True


# Implementation 2

In [17]:
ndfa2 = RegularExpressionGenerator(empty_symbol = "lambda")
ndfa2.load_from_file("automaton_examples/automata4.txt")

In [18]:
ndfa2.convert_automaton()

In [19]:
ndfa2.re_compiler

{('new_q0', 'new_final'): '(0*1)'}

In [20]:
ndfa2.re_expressions

{('new_q0', 'new_final'): '(0)*.1'}

## Inherited Methods

In [21]:
ndfa2.process_word("0000010", False)

False

In [22]:
ndfa2.process_word("000001", False)

True

In [23]:
ndfa2.print_valid_path()

1.- ('q0', '0')  ======> q0---- 
2.- ('q0', '0')  ======> q0---- 
3.- ('q0', '0')  ======> q0---- 
4.- ('q0', '0')  ======> q0---- 
5.- ('q0', '0')  ======> q0---- ('q0', 'lambda')  ======> q1---- 
6.- ('q1', '1')  ======> q2---- ('q2', 'lambda')  ======> q3---- 


### Re_compiler

In [24]:
string = "010011000010002100011hola2310001" 
ans = ndfa2.check_word(string)

01
001
1
00001
1
0001
1
1
0001


In [25]:
ans = list(set(ans))

In [26]:
for i in ans:
    print(ndfa2.process_word(i, False))

True
True
True
True
True


# Implementation 3

In [27]:
ndfa3 = RegularExpressionGenerator(empty_symbol = "lambda")
ndfa3.load_from_file("automaton_examples/automata3.txt")

In [28]:
ndfa3.convert_automaton()

In [29]:
ndfa3.re_compiler

{('new_q0',
  'new_final'): '((((0)?1*0)?(0|((1)?1*0))*)|(((0|((0)?1*))|(((0)?1*0)?(0|((1)?1*0))*((1)?1*)))(((0|0)|((0)?1*))|(((0)?1*0)?(0|((1)?1*0))*((1)?1*)))*(((0)?1*0)?(0|((1)?1*0))*)))'}

In [30]:
ndfa3.re_expressions

{('new_q0',
  'new_final'): '((λ U ((0 U λ).(1)*.0)).(0 U ((1 U λ).(1)*.0))* U (((0 U ((0 U λ).(1)*.λ)) U ((λ U ((0 U λ).(1)*.0)).(0 U ((1 U λ).(1)*.0))*.(1 U λ).(1)*)).(((0 U λ.0) U ((0 U λ).(1)*.λ)) U ((λ U ((0 U λ).(1)*.0)).(0 U ((1 U λ).(1)*.0))*.(1 U λ).(1)*))*.(λ U ((0 U λ).(1)*.0)).(0 U ((1 U λ).(1)*.0))*))'}

In [31]:
ndfa3.process_word("01", False)

True

In [32]:
string = "0100110210011hola2310001" 
ans = ndfa3.check_word(string, True)

0100110

100

11






1000

1



In [33]:
ans = list(set(ans))

In [34]:
ans

['', '0100110', '100', '1', '11', '1000']

In [35]:
for i in ans:
    print(ndfa3.process_word(i, False))

True
True
True
True
True
True


### Note: perpahs is weird that an empty string is part of the language, but it is you can check the automata and see there is empty transictions from the initial state to the final state 

## Implementation 4

In [36]:
ndfa4 = RegularExpressionGenerator(empty_symbol = "lambda")
ndfa4.load_from_file("automaton_examples/automata1.txt")

In [37]:
ndfa4.convert_automaton()

In [38]:
ndfa4.re_compiler

{('new_q0',
  'new_final'): '(((0|1)1*1)|((1|((0|1)1*(0|1)))((1)?|(01*(0|1)))*(0|(01*1))))'}

In [39]:
ndfa4.re_expressions

{('new_q0',
  'new_final'): '((0 U 1).(1)*.1 U ((1 U ((0 U 1).(1)*.(0 U 1))).(1 U λ U (0.(1)*.(0 U 1)))*.(0 U (0.(1)*.1))))'}

In [40]:
ndfa4.process_word("10101", False)

True

In [41]:
string = "010011000010002100011hola2310001" 
ans = ndfa4.check_word(string, True)

01
00110
000
10
10
011
10
01


In [42]:
for i in list(set(ans)):
    print(ndfa4.process_word(i, False))

True
True
True
True
True


# Implementation 5

Multiple final states

In [43]:
ndfa5 = RegularExpressionGenerator(empty_symbol = "lambda")
ndfa5.load_from_file("automaton_examples/automata5.txt")

In [44]:
ndfa5.f

['q2', 'q3']

In [45]:
ndfa5.convert_automaton()

In [46]:
ndfa5.re_compiler

{('new_q0',
  'new_final'): '(((b|((a|b)b*(a|b)))((b)?|(ab*(a|b)))*)|(((a|b)b*b)|((b|((a|b)b*(a|b)))((b)?|(ab*(a|b)))*(a|(ab*b)))))'}

In [47]:
ndfa5.re_expressions

{('new_q0',
  'new_final'): '((b U ((a U b).(b)*.(a U b))).(b U λ U (a.(b)*.(a U b)))* U ((a U b).(b)*.b U ((b U ((a U b).(b)*.(a U b))).(b U λ U (a.(b)*.(a U b)))*.(a U (a.(b)*.b)))).λ)'}

In [61]:
ndfa5.process_word("aaaa", False)

True

In [49]:
ndfa5.print_valid_path()

1.- ('q0', 'a')  ======> q1---- 
2.- ('q1', 'a')  ======> q2---- 
3.- ('q2', 'a')  ======> q1---- 
4.- ('q1', 'a')  ======> q2---- 
5.- ('q2', 'b')  ======> q2---- 
6.- ('q2', 'a')  ======> q1---- 
7.- ('q1', 'b')  ======> q2---- 
8.- ('q2', 'b')  ======> q2---- 
9.- ('q2', 'b')  ======> q2---- 
10.- ('q2', 'a')  ======> q1---- 
11.- ('q1', 'a')  ======> q2---- 
12.- ('q2', 'a')  ======> q3---- 


In [63]:
string = "aaaaaaaababababbbbaa" 
ans = ndfa5.check_word(string, True)

aa
aa
aa
aab
abab
abbbba


In [51]:
for i in list(set(ans)):
    print(ndfa5.process_word(i, False))

True
True
True
True


#### Nota: Profe en este problema nos dimos cuenta de algo, en la biblioteca al usar | que es un o, no siempre se encuentran todos los patrones, por ejemplo, en este problema los impares excepto 1 deben ser aceptados, como 3, 5, y así, pero solo encuentra los pares, y los separa de 2 en dos, porque el lado izquierdo del | hace match con los pares, y la biblioteca re al encontrar el patrón en la izquierda ya no prueba el de la derecha, entonces son cosas que pueden afectar este código, y la verdad no encontré solución, si pudiera decirme la solución se lo agredecería :). Abajo está el como hace match con los impares porque en el | intercambie los lados, pero ahora los pares después del 3 no aparecen y en impares solo aperecen de 3 en 3.

In [52]:
def match_patterns(pattern, string):
    match = re.findall(pattern, string)
    for match in re.finditer(pattern, string):
        print(match.group())

In [64]:
pattern = "((((a|b)(b)*b)|((b|((a|b)(b)*(a|b)))((b)?|(a(b)*(a|b)))*(a|(a(b)*b))))|((b|((a|b)(b)*(a|b)))((b)?|(a(b)*(a|b)))*))" 
string = "aaaa"
match_patterns(pattern, string)

aaa


In [65]:
pattern = "((((a|b)(b)*b)|((b|((a|b)(b)*(a|b)))((b)?|(a(b)*(a|b)))*(a|(a(b)*b))))|((b|((a|b)(b)*(a|b)))((b)?|(a(b)*(a|b)))*))" 
string = "aaaaa"
match_patterns(pattern, string)

aaa
aa


# Implementation 6

In [55]:
ndfa6 = RegularExpressionGenerator(empty_symbol = "epsilon")
ndfa6.load_from_file("automaton_examples/automata2.txt")

In [56]:
ndfa6.convert_automaton()

In [57]:
ndfa6.re_compiler

{}

In [58]:
ndfa6.re_expressions

{}

In [59]:
ndfa6.process_word("aba", False)

False

In [60]:
string = "010011000010002100011hola2310001" 
ans = ndfa6.check_word(string, False)

Exception: This automata its final states are not reacheable, so there is not a regular expression